# Exercise 1
Implement KNN by hand for just 2 dimensions with normalization.

### Create Dataframe
Zum Implementieren des KNN soll zunächst der Datensatz des Film-Beispiels aus dem Script als DataFrame definiert werden. Dieser Datensatz erhält auch einen siebten derzeit nicht kategorisierten Film, dessen "kicks" und "kisses" jedoch bekannt sind. Dieser Film soll im Folgenden durch die Nutzung von KNN kategorisiert werden und damit eine Klasse "Romance" oder "Action" erhalten.

In [1]:
import pandas as pd

movies = [
    ['California Man', 3, 104, 'Romance'],
    ['He\'s not really into dudes', 2, 100, 'Romance'],
    ['Beautiful Woman', 1, 81, 'Romance'],
    ['Kevin Longblade', 101, 10, 'Action'],
    ['Robo Slayer 3000', 99, 5, 'Action'],
    ['Amped', 98, 2, 'Action'],
    ['N/A', 73, 50, 'N/A'], #Unknown movie with missing genre
]

dataset = pd.DataFrame(movies, columns=['title', 'kicks', 'kisses', 'genre'])

dataset

,title,kicks,kisses,genre
0,California Man,3,104,Romance
1,He's not really into dudes,2,100,Romance
2,Beautiful Woman,1,81,Romance
3,Kevin Longblade,101,10,Action
4,Robo Slayer 3000,99,5,Action
5,Amped,98,2,Action
6,N/A,73,50,N/A


### Int to Float Conversion
Um die Werte der gegebenen Datentabelle im Folgenden einheitlich nutzen zu können, werden die Integer-Werte der Tabelle zunächst zu Float konvertiert.

In [2]:
dataset['kicks'] = dataset['kicks'].astype(float)
dataset['kisses'] = dataset['kisses'].astype(float)

dataset

,title,kicks,kisses,genre
0,California Man,3.0,104.0,Romance
1,He's not really into dudes,2.0,100.0,Romance
2,Beautiful Woman,1.0,81.0,Romance
3,Kevin Longblade,101.0,10.0,Action
4,Robo Slayer 3000,99.0,5.0,Action
5,Amped,98.0,2.0,Action
6,N/A,73.0,50.0,N/A


### Normalize Data in Another Table
Zunächst sollen die Werte der Spalten "kicks" und "kisses" normalisiert werden, um in weiteren Schritten mit diesem Arbeiten zu können. Somit wird eine Normalisierungs-Funktion definiert: 
$$normalized = \frac{value - column_{min}}{column_{max} - column_{min}}$$
Diese wird im Anschluss dafür genutzt, um jede Zelle der beiden Spalten zu normalisieren. Die so errechneten Werte werden in einer eigenen Tabelle als Werte für "kicks" und "kisses" eingesetzt.

In [3]:
def normalize(number, column):
    return (number - dataset[column].min()) / (dataset[column].max() - dataset[column].min())

normalized_data = dataset.copy()

for index, row in normalized_data.iterrows():
    normalized_data.loc[index, 'kicks'] = normalize(normalized_data.at[index, 'kicks'], "kicks")
    normalized_data.loc[index, 'kisses'] = normalize(normalized_data.at[index, 'kisses'], "kisses")
        
normalized_data

,title,kicks,kisses,genre
0,California Man,0.02,1.000000,Romance
1,He's not really into dudes,0.01,0.960784,Romance
2,Beautiful Woman,0.00,0.774510,Romance
3,Kevin Longblade,1.00,0.078431,Action
4,Robo Slayer 3000,0.98,0.029412,Action
5,Amped,0.97,0.000000,Action
6,N/A,0.72,0.470588,N/A


### Euclid Distance
Im Folgenden soll nun die Funktion zur Berechnung des Euklidischen Abstands umgesetzt werden. Die umzusetzende Gleichung lautet: 

$$ \sqrt{(P1_x - P2_x) + (P1_y - P2_y)}  $$

Da für die bewältigung der Aufgabe jedoch Helferklassen von Python weitestgehend umgangen werden sollen und für das Ziehen einer Wurzel numpy genutzt werden müsste, wird die Euklidische Abstandsfunktion ohne Wurzel wie folgt verwirklicht: 

$$ ((P1_x - P2_x) + (P1_y - P2_y))^\frac{1}{2} $$

In [4]:
def euclid(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**(1/2)

### Calculate the Distance
Mit Hilfe der erstellten Funktion zur Berechnung des Euklidischen Abstands kann nun eine neue Spalte in der vorhandenen Tabelle angelegt werden. Diese sammelt die errechneten Distanzen. Erwartungsweise besitzt der unbekannte Film eine Distanz von 0 zu sich selber.

In [5]:
missing_movie = [normalized_data["kicks"].iloc[-1], normalized_data["kisses"].iloc[-1]]

for index, row in normalized_data.iterrows():
    test_point = [normalized_data.at[index, 'kicks'], normalized_data.at[index, 'kisses']]
    normalized_data.loc[index, 'distance'] = euclid(missing_movie, test_point)
    
normalized_data

,title,kicks,kisses,genre,distance
0,California Man,0.02,1.000000,Romance,0.877654
1,He's not really into dudes,0.01,0.960784,Romance,0.862782
2,Beautiful Woman,0.00,0.774510,Romance,0.781517
3,Kevin Longblade,1.00,0.078431,Action,0.481858
4,Robo Slayer 3000,0.98,0.029412,Action,0.512090
5,Amped,0.97,0.000000,Action,0.532873
6,N/A,0.72,0.470588,N/A,0.000000


### Sort by Distance
Ans nächstes werden die ermittelten Distanzen zum Zielpunkt absteigend sortiert. Da der gesuchte Film per Definition einen Abstand von 0 zu sich hat, muss dieser somit an der Spitz der Tabelle stehen - gefolgt von den errechneten Distanzen der anderen Filme.

In [6]:
normalized_data = normalized_data.sort_values('distance')

normalized_data

,title,kicks,kisses,genre,distance
6,N/A,0.72,0.470588,N/A,0.000000
3,Kevin Longblade,1.00,0.078431,Action,0.481858
4,Robo Slayer 3000,0.98,0.029412,Action,0.512090
5,Amped,0.97,0.000000,Action,0.532873
2,Beautiful Woman,0.00,0.774510,Romance,0.781517
1,He's not really into dudes,0.01,0.960784,Romance,0.862782
0,California Man,0.02,1.000000,Romance,0.877654


### Aggregate by Target Variable
In einem nächsten Schritt soll anschließend das Ergebnis der Distanzberechnungen genutzt werden, um für ein ausgewähltes k die dichtesten Nachbarn zu finden. Hierfür wird eine allgemeine Funktion aufgestellt, die k-Mal das Genre der niedrigsten Distanzen wiedergibt.

In [7]:
def neighbors(k):
    return normalized_data.sort_values('distance').iloc[1:k + 1]

### Determine Target Class
Mit diesen Nachbarn ist es möglich zu ermitteln welche Target Class der Film haben soll. Hierfür wird ein k von 2 gewählt. Dies ist in Anbetracht des kleinen Datensatzes weniger als 50%. Zusätzlich wurde ein k höher als 1 gewählt, um zu verhindern, dass das Ergebnis auf nur einer Datengrundlage getroffen wird.

In [8]:
final = neighbors(2)

action = 0
romance = 0
for i, neighbor in final.iterrows():
    if (neighbor["genre"] == "Action"):
        action = action + 1
    elif (neighbor["genre"] == "Romance"):
        romance = romance + 1
    

print("Votes: Action: ", action, "| Romance: ", romance)

Votes: Action:  2 | Romance:  0


Aus der Auswertung geht somit hervor, dass die zwei dichtesten Nachbarn dem Genre "Action" zuzuordnen sind. Damit kann die Target Class des Films ohne Genre als "Action" definiert werden.